In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163477 sha256=e6b2a4de2f33dead2b2e1bdaa1407d71b5dd51053f01b983c1ea25111683e939
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action = 'ignore')
from matplotlib import font_manager, rc
import platform

from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from surprise import dump
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)

In [4]:
all = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/1. 하이브리드 모델/전처리 파일/c_score.csv', encoding='cp949')

In [ ]:
# 결측치 확인
all.isna().sum()

subsr             0
series_nm         0
Affinity          0
ct_cl             0
genre_of_ct_cl    0
dtype: int64

In [5]:
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7711 entries, 0 to 7710
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   subsr           7711 non-null   int64  
 1   series_nm       7711 non-null   object 
 2   Affinity        7711 non-null   float64
 3   ct_cl           7711 non-null   object 
 4   genre_of_ct_cl  7711 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 301.3+ KB


In [6]:
from sklearn.preprocessing import MinMaxScaler
# 로그 변환 적용
all['Affinity'] = np.log1p(all['Affinity'])

# Min-Max 스케일링 적용
scaler = MinMaxScaler(feature_range=(1, 5))
all['Affinity'] = scaler.fit_transform(all[['Affinity']])

# 스케일링된 값 정수로 변환
all['Affinity'] = all['Affinity'].round().astype(int)

# 결과 확인
all.head()

,subsr,series_nm,Affinity,ct_cl,genre_of_ct_cl
0,59879000,나쁜엄마,1,TV드라마,기타
1,59879000,나의 해방일지,1,TV드라마,기타
2,59879000,낭만닥터 김사부2,1,TV드라마,기타
3,59879000,소방서 옆 경찰서,2,TV드라마,기타
4,59879000,슬기로운 의사생활 시즌2,1,TV드라마,기타


In [ ]:
all[all["Affinity"] == 4]

,subsr,series_nm,Affinity,ct_cl,genre_of_ct_cl
106,59933000,런닝맨,4,TV 연예/오락,기타
206,60067000,소녀대인,4,TV드라마,외화 시리즈
210,60067000,"암련, 귤생회남",4,TV드라마,외화 시리즈
216,60067000,전문중적진천천,4,TV드라마,외화 시리즈
399,60326000,구소한야난 너의 온도,4,TV드라마,외화 시리즈
413,60326000,무림유협기,4,TV드라마,외화 시리즈
442,60326000,유리미인살,4,TV드라마,외화 시리즈
446,60326000,장야 2,4,TV드라마,외화 시리즈
455,60326000,천고결진,4,TV드라마,외화 시리즈
469,60326000,호심,4,TV드라마,외화 시리즈


In [7]:
# train, test 나누기
len(all)
cutoff = int(.5 * len(all))
train_r = all[:cutoff]
test_r = all[cutoff:]

train_r = train_r[['subsr', 'series_nm', 'Affinity', 'ct_cl', 'genre_of_ct_cl']]
test_r = test_r[['subsr', 'series_nm', 'Affinity', 'ct_cl', "genre_of_ct_cl"]]
print("train_size:", len(train_r), "test_size:", len(test_r))
train_r.head()

train_size: 3855 test_size: 3856


,subsr,series_nm,Affinity,ct_cl,genre_of_ct_cl
0,59879000,나쁜엄마,1,TV드라마,기타
1,59879000,나의 해방일지,1,TV드라마,기타
2,59879000,낭만닥터 김사부2,1,TV드라마,기타
3,59879000,소방서 옆 경찰서,2,TV드라마,기타
4,59879000,슬기로운 의사생활 시즌2,1,TV드라마,기타


In [8]:
all["Affinity"].value_counts()

1    5447
2    1897
3     305
4      60
5       2
Name: Affinity, dtype: int64

In [9]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    # rating 값의 최소 및 최대값을 지정
    reader = Reader(rating_scale=(1, 5))

    # 데이터프레임에서 Surprise의 데이터셋 형식으로 변환
    trainset = Dataset.load_from_df(training_dataframe[['subsr', 'series_nm', 'Affinity']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['subsr', 'series_nm', 'Affinity']], reader)
    # Surprise에서 사용하는 데이터셋 형식으로 변환
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)

    return trainset, testset

In [10]:
trainset, testset = convert_traintest_dataframe_forsurprise(train_r, test_r)

In [21]:
trainset

## Knn-baseline 모델

In [11]:
# content-based filtering (CBF)
# 옵션 설정 : cosine 유사도를 이용, user_based가 False면 content(item)-based similarity 사용
sim_options = {'name': 'cosine',
               'user_based': False  # content(item)-based similarity
               }

# Surprise 라이브러리의 KNNBaseline 알고리즘에 앞의 옵션 적용
knnbaseline_algo_i = KNNBaseline(sim_options=sim_options)

knnbaseline_algo_i.fit(trainset)
# knnbaseline_predictions_u : 테스트로 수행한 예측 결과
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)

file_name = 'KnnBaseline_model_i'
dump.dump(file_name, algo=knnbaseline_predictions_i)

accuracy.rmse(knnbaseline_predictions_i)
accuracy.mae(knnbaseline_predictions_i)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.5631
MAE:  0.4719


0.4719094773185179

In [12]:
# user-based filtering (CBF)
sim_options = {'name': 'cosine',
               'user_based': True  # user-based similarity
               }

# Surprise 라이브러리의 KNNBaseline 알고리즘에 앞의 옵션 적용
knnbaseline_algo_u = KNNBaseline(sim_options=sim_options)

knnbaseline_algo_u.fit(trainset)
# knnbaseline_predictions_u : 테스트로 수행한 예측 결과
knnbaseline_predictions_u = knnbaseline_algo_u.test(testset)

file_name = 'KnnBaseline_model_u'
dump.dump(file_name, algo=knnbaseline_predictions_u)

accuracy.rmse(knnbaseline_predictions_u)
accuracy.mae(knnbaseline_predictions_u)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.5635
MAE:  0.4716


0.4715634608583271

In [13]:
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)
knnbaseline_predictions_i[:3]

[Prediction(uid=64385000, iid='식객 허영만의 백반기행', r_ui=3.0, est=1.60664277669596, details={'was_impossible': False}),
 Prediction(uid=64385000, iid='신들의 사생활: 그리스 로마신화 2', r_ui=2.0, est=1.60664277669596, details={'was_impossible': False}),
 Prediction(uid=64385000, iid='신랑수업', r_ui=4.0, est=2.004102586083216, details={'actual_k': 1, 'was_impossible': False})]

In [14]:
# uid(사용자 id), iid(아이템 id), r_ui(실제 평점) ,est(예측평점) 추출

[(pred.uid, pred.iid, pred.r_ui, pred.est) for pred in knnbaseline_predictions_i[:3]]

[(64385000, '식객 허영만의 백반기행', 3.0, 1.60664277669596),
 (64385000, '신들의 사생활: 그리스 로마신화 2', 2.0, 1.60664277669596),
 (64385000, '신랑수업', 4.0, 2.004102586083216)]

In [15]:
# uid = 998923 에 대한 유사도 높은 예측 iid, est 조회.
# 참고: https://hmkim312.github.io/posts/%EC%84%9C%ED%94%84%EB%9D%BC%EC%9D%B4%EC%A6%88(Surprise)%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%9C_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommendations)/
pred_df = pd.DataFrame(knnbaseline_predictions_i)
pred_df[(pred_df['uid']==64385000)]

,uid,iid,r_ui,est,details
0,64385000,식객 허영만의 백반기행,3.0,1.606643,{'was_impossible': False}
1,64385000,신들의 사생활: 그리스 로마신화 2,2.0,1.606643,{'was_impossible': False}
2,64385000,신랑수업,4.0,2.004103,"{'actual_k': 1, 'was_impossible': False}"
3,64385000,신발 벗고 돌싱포맨,1.0,2.180097,"{'actual_k': 27, 'was_impossible': False}"
4,64385000,신비한TV서프라이즈,1.0,1.606643,{'was_impossible': False}
5,64385000,신상 출시 편스토랑,4.0,1.606643,{'was_impossible': False}
6,64385000,실화탐사대,2.0,2.120859,"{'actual_k': 21, 'was_impossible': False}"
7,64385000,싱포골드,1.0,1.606643,{'was_impossible': False}
8,64385000,아는 형님,3.0,1.839005,"{'actual_k': 27, 'was_impossible': False}"
9,64385000,안싸우면 다행이야,1.0,1.946691,"{'actual_k': 23, 'was_impossible': False}"


 ## svd, svdpp (CF) 모델

In [16]:
 # svd : 특이값 분해를 기반으로 수행
 # 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#matrix-factorization-based-algorithms

# Surprise 라이브러리에서 제공하는 SVD알고리즘의 기본 설정을 이용해서 알고리즘 객체 생성
svd_algo = SVD()

svd_algo.fit(trainset)
# svd_predictions : 테스트로 수행한 예측 결과
svd_predictions = svd_algo.test(testset)

file_name = 'svd_model'
dump.dump(file_name, algo=svd_algo)

accuracy.rmse(svd_predictions)
accuracy.mae(svd_predictions)
print("Done!")

RMSE: 0.5613
MAE:  0.4679
Done!


In [23]:
# matrix factorization/ collaborative filtering(CF)
# svd++ model : implicit ratings까지 계산
# 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp


#Surprise 라이브러리에서 제공하는 SVD++ 알고리즘의 기본 설정을 사용
svdpp_algo = SVDpp()

svdpp_algo.fit(trainset)
svdpp_predictions = svdpp_algo.test(testset)

file_name = 'svdpp_model'
dump.dump(file_name, algo=svdpp_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svdpp_predictions)
accuracy.mae(svdpp_predictions)
print("Done!")

RMSE: 0.5619
MAE:  0.4674
Done!


## gridsearchCV 모델

In [24]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import GridSearchCV

# Surprise 라이브러리에서 모델의 하이퍼파라미터를 튜닝하기 위한 그리드 서치(Grid Search)를 수행하기 위한 파라미터 그리드를 정의하는 부분
# n_epochs : SGD (Stochastic Gradient Descent)의 에폭 수를 5번과 20번으로 반복 수행
# 에폭 수 : 전체 데이터셋에 대해 한 번 학습을 완료하는 주기
# lr_all : 모든 매개변수에 대한 학습률로, 각 반복에서 매개변수가 얼마나 조정되는지 결정하는 매개변수
# lr_all : 가중치를 0.002와 0.005 두가지로 설정
# reg_all : 정규화 항의 크기 (모델이 복잡해지는 것을 제어, 일반화 성능을 향상) 0.4와 0.6을 테스트
param_grid = {'n_epochs': [5, 20], 'lr_all': [0.002, 0.005],'reg_all': [0.4, 0.6]}

# 성능지표로 rmse와 mae 사용
# cv : 교차 검증을 5개 폴드로 설정, 데이터를 5등분하여 5번에 걸쳐 학습과 평가를 반복하며 모델 성능을 측정
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)

reader = Reader()

# The columns must correspond to user id, item id and ratings (in that order).
# Surprise의 Dataset 형식으로 변환, reader로 데이터를 읽고 처리
train_data = Dataset.load_from_df(train_r[['subsr', 'series_nm', 'Affinity']], reader)

gs.fit(train_data)

# 최적의 rmse 값 도출
print("gs.best_score(rmse): ", gs.best_score['rmse'])

# 최적의 rmse 값의 파라미터 도출
print("gs.best_params: ", gs.best_params['rmse'])

gs.best_score(rmse):  0.563508255989601
gs.best_params:  {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}


In [26]:
# 최적의 모델 적용
GS_SVDpp_best = gs.best_estimator['rmse']

# train_data를 Surprise 모델이 학습할 수 있는 형태의 데이터(trainset)로 구조화
trainset = train_data.build_full_trainset()
GS_SVDpp_best.fit(trainset)

# validation set으로 테스트
predictions = GS_SVDpp_best.test(trainset.build_testset())
print('validation test,', end='\n')
accuracy.rmse(predictions)
accuracy.mae(predictions)
# 참고:https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-save-some-data-for-unbiased-accuracy-estimation
## 참고: https://surprise.readthedocs.io/en/stable/getting_started.html#train-on-whole-trainset


validation test,
RMSE: 0.5005
MAE:  0.4091


0.4091181605191966

In [27]:
# Compute biased accuracy on train (validation set)
predictions = GS_SVDpp_best.test(testset)
print('testset accuracy,', end='\n')
accuracy.rmse(predictions)
accuracy.mae(predictions)

testset accuracy,
RMSE: 0.5619
MAE:  0.4721


0.4721167653510425

In [28]:
# k-fold trainset, testset 만드는 함수
# k-fold란 교차 검증을 수행할 때 사용되는 학습 데이터의 하위 집합
# 교차 검증 : 모델을 성능 평가를 위해 데이터를 여러 부분으로 나누고,  훈련 데이터셋과 검증 데이터 셋으로 이용하는 것
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import KFold

reader = Reader()
data = Dataset.load_from_df(all[['subsr', 'series_nm', 'Affinity']], reader)

# define a cross-validation iterator
kf = KFold(n_splits=5)

def train_test_get(data, gettype):
  for trainset, testset in kf.split(data):
    if gettype == "train":
      return trainset
    elif gettype == "test":
      return testset

# train_test_get(data, "test")

In [30]:
# testset
pred_testset = GS_SVDpp_best.test(train_test_get(data, "test"))

In [31]:
# cv_results
results = pd.DataFrame.from_dict(gs.cv_results)
results.T

,0,1,2,3,4,5,6,7
split0_test_rmse,0.597051,0.598182,0.581649,0.582811,0.57319,0.575032,0.560997,0.563635
split1_test_rmse,0.593218,0.591742,0.578548,0.579755,0.572748,0.574065,0.568097,0.570104
split2_test_rmse,0.602806,0.601903,0.589231,0.588601,0.580868,0.584007,0.575167,0.577341
split3_test_rmse,0.551332,0.552843,0.539903,0.541922,0.538488,0.540586,0.536432,0.535851
split4_test_rmse,0.604065,0.605409,0.588752,0.590549,0.582008,0.584363,0.576847,0.579142
mean_test_rmse,0.589694,0.590016,0.575616,0.576727,0.56946,0.571611,0.563508,0.565215
std_test_rmse,0.019579,0.01913,0.018321,0.01783,0.015947,0.016103,0.014658,0.015682
rank_test_rmse,7,8,5,6,3,4,1,2
split0_test_mae,0.494002,0.496982,0.481425,0.484123,0.476685,0.478731,0.462813,0.467009
split1_test_mae,0.49174,0.492236,0.477385,0.481851,0.472111,0.47525,0.464584,0.468817


## evaluation (knn/svd/svdpp/gridsearchCV 비교)

In [33]:
knn_i_pred = knnbaseline_algo_i.test(testset)
knn_u_pred = knnbaseline_algo_u.test(testset)
svd_pred = svd_algo.test(testset)
svdpp_pred = svdpp_algo.test(testset)
gridCV_svdpp_pred = GS_SVDpp_best.test(testset)

In [34]:
# test_rmse, test_mae
knn_i_rmse = accuracy.rmse(knn_i_pred)
knn_i_mae = accuracy.mae(knn_i_pred)
knn_u_rmse = accuracy.rmse(knn_u_pred)
knn_u_mae = accuracy.mae(knn_u_pred)
svd_rmse = accuracy.rmse(svd_pred)
svd_mae = accuracy.mae(svd_pred)
svdpp_rmse = accuracy.rmse(svdpp_pred)
svdpp_mae = accuracy.mae(svdpp_pred)
gridCV_svdpp_rmse = accuracy.rmse(gridCV_svdpp_pred)
gridCV_svdpp_mae = accuracy.mae(gridCV_svdpp_pred)

RMSE: 0.5631
MAE:  0.4719
RMSE: 0.5635
MAE:  0.4716
RMSE: 0.5613
MAE:  0.4679
RMSE: 0.5619
MAE:  0.4674
RMSE: 0.5619
MAE:  0.4721


### recall & precision @ k
참고 : https://surprise.readthedocs.io/en/stable/FAQ.html?highlight=recall#how-to-compute-precision-k-and-recall-k


In [35]:
from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # 각 유저별 예측 평점(est)과 실제 평점(true_r)의 쌍을 저장
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # 각 사용자의 예측 평점을 내림차순으로 정렬
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # n_rel, n_rec_k, n_rel_and_rec_k는 관계있는 아이템 수 top k에서
        # 추천된 아이템 수, 그리고 top k에서 관계있는 아이템 수를 나타냄

        # 관련 있는 아이템 수
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # top k에 있는 추천 아이템 수
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [37]:
# knn_i 모델 precision&recall
knn_i_prec, knn_i_re = precision_recall_at_k(knn_i_pred, k=5, threshold=4)

# Precision & recall
knn_i_prec_avg = sum(prec for prec in knn_i_prec.values()) / len(knn_i_prec)
print(knn_i_prec_avg)
knn_i_re_avg = sum(rec for rec in knn_i_re.values()) / len(knn_i_re)
print(knn_i_re_avg)

0.0
0.0


In [38]:
# knn_u 모델 precision&recall
knn_u_prec, knn_u_re = precision_recall_at_k(knn_u_pred, k=5, threshold=4)

# Precision & recall
knn_u_prec_avg = sum(prec for prec in knn_u_prec.values()) / len(knn_u_prec)
print(knn_u_prec_avg)
knn_u_re_avg = sum(rec for rec in knn_u_re.values()) / len(knn_u_re)
print(knn_u_re_avg)

0.0
0.0


In [39]:
# svd precision&recall
svd_prec, svd_re = precision_recall_at_k(svd_pred, k=5, threshold=4)

# Precision & recall
svd_prec_avg = sum(prec for prec in svd_prec.values()) / len(svd_prec)
print(svd_prec_avg)
svd_re_avg = sum(rec for rec in svd_re.values()) / len(svd_re)
print(svd_re_avg)

0.0
0.0


In [40]:
# svdpp precision&recall
svdpp_prec, svdpp_re = precision_recall_at_k(svdpp_pred, k=5, threshold=4)

# Precision & recall
svdpp_prec_avg = sum(prec for prec in svdpp_prec.values()) / len(svdpp_prec)
print(svdpp_prec_avg)
svdpp_re_avg = sum(rec for rec in svdpp_re.values()) / len(svdpp_re)
print(svdpp_re_avg)

0.0
0.0


In [41]:
# gridCV_svdpp precision&recall

grid_prec, grid_re = precision_recall_at_k(gridCV_svdpp_pred, k=5, threshold=4)

# Precision & recall
grid_prec_avg = sum(prec for prec in grid_prec.values()) / len(grid_prec)
print(grid_prec_avg)
grid_re_avg = sum(rec for rec in grid_re.values()) / len(grid_re)
print(grid_re_avg)

0.0
0.0


### model별 test_rmse/mae/precision/recall 테이블

In [42]:
# model별 test prediction  데이터
# row: Knnbaseline(item based), Knnbaseline(user based), SVD(CF), SVDpp(CF), gridsearch(SVDpp)
# columns: test RMSE, test MAE, test_time, Precision, recall
# hybrid 모델 수치는 SVDpp(CF)모델과 Knnbaseline(item-based)의 평균값.
test_prediction = pd.DataFrame({"Algorithm": ["KnnBaseline(item-based)" ,
                                              "KnnBaseline(user-based)",
                                              "SVD(CF)",
                                              "SVDpp(CF)",
                                              "gridsearch(SVDpp)",
                                              "Hybrid(CF+CBF)"

                                              ]
                                ,"test_rmse": [knn_i_rmse , knn_u_rmse, svd_rmse , svdpp_rmse , gridCV_svdpp_rmse, svdpp_rmse*0.4+knn_i_rmse*0.6]
                                ,"test_mae":[knn_i_mae , knn_u_mae, svd_mae, svdpp_mae , gridCV_svdpp_mae, svdpp_mae*0.4+knn_i_mae*0.6]
                                ,"Precision":[knn_i_prec_avg ,knn_u_prec_avg ,svd_prec_avg ,svdpp_prec_avg , grid_prec_avg, svdpp_prec_avg*0.4+knn_i_prec_avg*0.6]
                                ,"recall":[knn_i_re_avg ,knn_u_re_avg ,svd_re_avg ,svdpp_re_avg ,grid_re_avg, svdpp_re_avg*0.4+knn_i_re_avg*0.6 ]
                                })

test_prediction

,Algorithm,test_rmse,test_mae,Precision,recall
0,KnnBaseline(item-based),0.563056,0.471909,0.0,0.0
1,KnnBaseline(user-based),0.563492,0.471563,0.0,0.0
2,SVD(CF),0.561307,0.467938,0.0,0.0
3,SVDpp(CF),0.561920,0.467431,0.0,0.0
4,gridsearch(SVDpp),0.561874,0.472117,0.0,0.0
5,Hybrid(CF+CBF),0.562602,0.470118,0.0,0.0
